# Graph Convolutional Network (GCN) Implementation

This notebook implements Graph Convolutional Networks (GCNs) for node classification tasks using the Cora citation network dataset.

## Setup and Data Loading

Import necessary libraries and load the Cora citation network dataset.

In [ ]:
# Core libraries
import torch
import torch.nn.functional as F
from torch.nn import Linear, Dropout

# PyTorch Geometric for graph neural networks
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

# Set random seed for reproducibility
torch.manual_seed(42)

In [ ]:
# Load Cora citation network dataset
dataset = Planetoid(root='.', name='Cora')
data = dataset[0]

In [ ]:
# Display dataset statistics
print("\n" + "="*50)
print(f"{('CORA DATASET STATISTICS'):^50}")
print("="*50)
print(f"Dataset Name:      {dataset.name}")
print(f"Number of Graphs:  {len(dataset):,}")
print(f"Number of Nodes:   {data.x.shape[0]:,}")
print(f"Number of Edges:   {data.edge_index.shape[1]:,}")
print(f"Node Features:     {dataset.num_features}")
print(f"Number of Classes: {dataset.num_classes}")
print(f"Train Nodes:       {data.train_mask.sum().item():,}")
print(f"Validation Nodes:  {data.val_mask.sum().item():,}")
print(f"Test Nodes:        {data.test_mask.sum().item():,}")
print("="*50)

## Utility Functions

Helper functions for model training and evaluation.

In [ ]:
def accuracy(y_pred, y_true):
    """Calculate classification accuracy"""
    return torch.sum(y_pred == y_true) / len(y_true)

## Graph Convolutional Network (GCN)

Implementation of the Graph Convolutional Network using PyTorch Geometric.

In [ ]:
class GCN(torch.nn.Module):
    """Graph Convolutional Network for node classification"""
    
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)  # First GCN layer
        self.gcn2 = GCNConv(dim_h, dim_out)  # Second GCN layer
        self.dropout = Dropout(0.5)  # Dropout for regularization
    
    def forward(self, x, edge_index):
        # First GCN layer with ReLU activation
        h = self.gcn1(x, edge_index)
        h = F.relu(h)
        h = self.dropout(h)
        
        # Second GCN layer
        h = self.gcn2(h, edge_index)
        return F.log_softmax(h, dim=1)
    
    def fit(self, data, epochs):
        """Train the GCN model"""
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        
        self.train()
        print("\n" + "="*60)
        print(f"{('Training Graph Convolutional Network'):^60}")
        print("="*60)
        print(f"{('Epoch'):>5} {('Train Loss'):>12} {('Train Acc'):>12} {('Val Loss'):>12} {('Val Acc'):>12}")
        print("-"*60)
        
        for epoch in range(epochs + 1):
            optimizer.zero_grad()
            out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
            loss.backward()
            optimizer.step()
            
            if epoch % 20 == 0:
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
                print(f"{epoch:5d} {loss.item():12.4f} {acc.item()*100:11.2f}% {val_loss.item():12.4f} {val_acc.item()*100:11.2f}%")
        
        print("-"*60)
    
    def test(self, data):
        """Evaluate on test set"""
        self.eval()
        out = self(data.x, data.edge_index)
        acc = accuracy(out[data.test_mask].argmax(dim=1), data.y[data.test_mask])
        return acc

## Model Training and Evaluation

Initialize, train and evaluate the GCN model.

In [ ]:
# Initialize GCN model
gcn = GCN(dataset.num_features, 16, dataset.num_classes)

print("\n" + "="*40)
print(f"{('GCN ARCHITECTURE'):^40}")
print("="*40)
print(gcn)
print(f"Total Parameters: {sum(p.numel() for p in gcn.parameters()):,}")
print("="*40)

In [ ]:
# Train GCN model
gcn.fit(data, 100)

# Test the model
gcn_test_acc = gcn.test(data)

print(f"\n{('='*30)}")
print(f"{('GCN FINAL RESULTS'):^30}")
print(f"{('='*30)}")
print(f"Test Accuracy: {gcn_test_acc.item()*100:6.2f}%")
print(f"{('='*30)}")